In [1]:
import numpy as np
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.arm_action_modes import ArmActionMode, JointVelocity, JointPosition, EndEffectorPoseViaPlanning, EndEffectorPoseViaIK


from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.environment import Environment
from rlbench.observation_config import ObservationConfig, CameraConfig
# from rlbench.tasks.pick_described_object import PickDescribedObject
from rlbench.tasks import PutGroceriesInCupboard, PickAndLift, StackBlocks, PlaceHangerOnRack, PickDescribedObject, TakeLidOffSaucepan, SetTheTable
from scipy.spatial.transform import Rotation as R
from matplotlib import pyplot as plt
from rlbench.sim2real.domain_randomization import RandomizationConfig


class Agent(object):
    def __init__(self, action_shape):
        self.action_shape = action_shape

    def act(self, obs):
        xyz = np.array([2.78478146e-01, -8.15162994e-03,  1.47194254e+00])
        rot = np.array([-3.14, 0.24243259,  3.14])
        #rot to quat
        quat = R.from_euler('xyz', rot).as_quat()
        gripper = [1.0]
        return np.concatenate([xyz,quat, gripper], axis=-1)


In [2]:
camera = CameraConfig(image_size=(224, 224), depth=False, point_cloud=False, mask=False)
obs_config = ObservationConfig(left_shoulder_camera=camera, right_shoulder_camera=camera, front_camera=camera, overhead_camera=camera)

env = Environment(
    action_mode=MoveArmThenGripper(
        arm_action_mode=EndEffectorPoseViaPlanning(absolute_mode=True, collision_checking=True), gripper_action_mode=Discrete()),
    obs_config=obs_config,
    robot_setup='panda',
    # randomize_every=1,
    # visual_randomization_config=RandomizationConfig(),
    # dynamics_randomization_config=RandomizationConfig(),
    headless=False)
env.launch()



In [14]:
target_item_poses = []
waypoints = []
gripper_poses = []
front_rgbs = []
def callable_fun(obs, task, variation_num):
    # target item pose
    target_item_pose = task._task.get_graspable_objects()[variation_num].get_pose()
    target_item_pose = np.concatenate([target_item_pose[:3], R.from_quat(target_item_pose[3:]).as_euler('xyz')])
    target_item_poses.append(target_item_pose)

    #waypoints
    current_pose = np.array(obs.gripper_pose)
    current_pose = np.concatenate([current_pose[:3], R.from_quat(current_pose[3:]).as_euler('xyz')])
    gripper_poses.append(current_pose)
    wps = [wp._waypoint.get_position() for wp in task._task._waypoints]
    
    if abs(current_pose[:3] - wps[0]).mean() < 1e-2:
        waypoints.append(0)
    elif abs(current_pose[:3] - wps[1]).mean() < 1e-2:
        waypoints.append(1)
    elif abs(current_pose[:3] - wps[2]).mean() < 1e-2:
        waypoints.append(2)
    else:
        waypoints.append(-1)

    #front rgb
    front_rgbs.append(obs.front_rgb)
    

variation_num = 1
task = env.get_task(PickDescribedObject)
task.set_variation(variation_num)


# demos = task.get_demos(1, live_demos=True, callable_each_step=lambda x: callable_fun(x,task=task, variation_num=variation_num)) 
# basket_position = task._task.dropin_box.get_position()
# demos = np.array(demos).flatten()
# descriptions, obs = task.reset() 

In [7]:
lower_bound = np.array([task._scene._workspace_minx, task._scene._workspace_miny, task._scene._workspace_minz])
upper_bound = np.array([task._scene._workspace_maxx, task._scene._workspace_maxy, task._scene._workspace_maxz])

In [17]:
task.reset()


In [21]:
np.random.uniform(-3.14, 3.14, 1)

array([1.43620341])

In [22]:
np.concatenate([[0,0],np.random.uniform(-3.14, 3.14, 1)])

array([ 0.        ,  0.        , -0.59670369])

In [23]:
while True:
    try:
        task.reset()
        [g.set_mass(10) for g in task._scene.task.get_graspable_objects()]
        pos = np.random.uniform(lower_bound, upper_bound)
        rot = np.concatenate([[0,0],np.random.uniform(-3.14, 3.14, 1)])
        joint_position = task._scene.robot.arm.solve_ik_via_sampling(position=pos, euler=rot)
        print(joint_position)
        task._scene.robot.arm.set_joint_positions(joint_position[0], disable_dynamics=True)
        task._scene.get_demo() 
    except Exception as e:
        print(e)
        continue

Could not find a valid joint configuration for desired end effector pose.
Could not find a valid joint configuration for desired end effector pose.
[[-0.24464975 -0.6636833   1.60429692 -2.36402249 -2.32371354  1.00386715
   2.22322917]]
[[ 1.90026295  0.69834369 -1.24154949 -1.5974555  -2.47537827  1.39533424
   0.88429606]]
Error in task pick_described_object. Demo was completed, but was not successful.
Could not find a valid joint configuration for desired end effector pose.
[[ 0.00967372 -1.49595559  0.49677411 -2.51521063 -2.5585649   2.09974694
  -1.66110706]]
Error in task pick_described_object. Demo was completed, but was not successful.
[[ 0.59480083  1.73567116 -1.69148958 -1.26195729 -1.37286305  1.62037659
  -2.13434911]]
[[ 1.29476559  0.8506071  -0.65145767 -1.66414869 -2.52634454  0.91050774
  -1.17557061]]
Error in task pick_described_object. Demo was completed, but was not successful.
[[-2.67647314 -0.95002103  2.75932956 -2.70044804 -0.68128598  0.50202042
   1.038598

KeyboardInterrupt: 

In [ ]:
pos

array([0.45840645, 0.11885708, 1.25888794])

In [ ]:
t[0]

array([-2.39029574, -1.04149449,  1.40677524, -0.87902278,  1.31014657,
        1.34261286,  0.62331313])

In [ ]:
task._task.robot.arm.get_joint_positions()

In [ ]:
task._task.robot.arm.set_joint_positions(t, disable_dynamics=False)

RuntimeError: Tried to set values for 1 joints, but joint group consists of 7 joints.

In [ ]:
pose = task._scene.robot.arm.get_pose()

In [ ]:
pose = task._scene.robot.gripper.get_pose()

In [ ]:
pose

array([-0.30895138,  0.        ,  0.82001764,  0.70493394, -0.05539087,
        0.70493394,  0.05539087])

In [ ]:
task._scene.robot.arm.get_configs_for_tip_pose(position=pose[:3], quaternion= pose[3:])

[array([ 1.71994376,  1.03063357,  0.76814634, -2.73857355, -0.62196368,
         1.4484781 ,  1.51707864]),
 array([ 1.78374875,  0.97574729,  0.74261177, -2.74671102, -0.58472776,
         1.48901153,  1.46639824]),
 array([ 1.57353914,  1.17086387,  0.81756955, -2.71960711, -0.70464647,
         1.3380208 ,  1.63778281]),
 array([ 1.89191079,  0.89694011,  0.69357264, -2.75655031, -0.52306801,
         1.54782355,  1.38681591]),
 array([ 1.90677536,  0.88686758,  0.6864962 , -2.75784469, -0.51455921,
         1.55495083,  1.37617207]),
 array([ 1.95952666,  0.85279727,  0.66018605, -2.76258612, -0.48476136,
         1.57879663,  1.33903444]),
 array([ 1.38128495,  1.38814032,  0.85802186, -2.68991566, -0.80224621,
         1.15676236,  1.80745387]),
 array([ 2.09896636,  0.77650785,  0.58535618, -2.77252078, -0.40806004,
         1.63110006,  1.24682999]),
 array([ 1.31514108,  1.46905863,  0.86454886, -2.67892432, -0.82945621,
         1.08683372,  1.86835229]),
 array([ 1.27473688

In [ ]:
task._scene.robot.arm.get_configs_for_tip_pose(position=pos, euler = rot)

ConfigurationError: Could not find a valid joint configuration for desired end effector pose.

In [ ]:
task._scene.robot.gripper.joints

array([0.53118789, 0.35893789, 1.11756635])

In [ ]:
waypoints.index(2)

198

In [ ]:
task._task._waypoints[0]._waypoint.get_pose()

array([ 3.22548747e-01,  1.10074833e-01,  8.24993849e-01, -1.66194528e-01,
       -9.86092985e-01, -1.77989335e-04,  1.81580181e-05])

In [ ]:
task._task.grasp_points[0].get_pose()

array([ 1.04694076e-01, -1.34871051e-01,  8.24995041e-01, -7.08412409e-01,
       -7.05798745e-01, -9.15655528e-06, -9.35975368e-06])

In [ ]:
task._task.get_graspable_objects()[0].get_pose()

array([ 1.04602322e-01, -1.59870893e-01,  7.99995065e-01, -9.98937821e-08,
       -1.30509416e-05, -1.84817833e-03,  9.99998331e-01])

In [ ]:
task._task.robot.gripper

In [ ]:
task._task._waypoints[0].get_waypoint_object().get_pose()
task.step(np.concatenate([task._task._waypoints[0].get_waypoint_object().get_pose(),[1]]))

(<rlbench.backend.observation.Observation at 0x7fa0a5d9d150>, 0.0, False)

In [ ]:
task.step(np.concatenate([task._task.target_block.get_pose(),[0]]))

(<rlbench.backend.observation.Observation at 0x7d01e17c2310>, 0.0, False)

In [ ]:
task.step(np.concatenate([task._task.target_block.get_pose(),[0]]))

In [ ]:
# task._task.distractors[0].get_pose()
task.step(np.concatenate([task._task.distractors[0].get_pose(),[0]]))

InvalidActionError: A path could not be found. Most likely due to the target being inaccessible or a collison was detected.

In [ ]:
task = env.get_task(PickAndLift)

agent = Agent(env.action_shape)
training_steps = 120
episode_length = 40
obs = None
for i in range(training_steps):
    # if i % episode_length == 0:
    print("Reset Episode")
    descriptions, obs = task.reset()
    print(descriptions)
    item_name = task._task.item_name
    item_coord = task._task.item.get_position()
    item_orientation = task._task.item.get_orientation()
    print(item_name)
    print(item_coord)
    print(item_orientation)
    action = agent.act(obs)
    obs, reward, terminate = task.step(action)
    print(reward)
    break


Reset Episode
['pick up the red block and lift it up to the target', 'grasp the red block to the target', 'lift the red block up to the target']


AttributeError: 'PickAndLift' object has no attribute 'item_name'

In [ ]:
task._task.get_waypoints()[0].get_waypoint_object().get_position()

array([ 0.13631016, -0.06427181,  1.20000005])

In [ ]:

print("Done")
env.shutdown()

Done
[CoppeliaSim:loadinfo]   CoppeliaSim ended.
